#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
## Building a word count application in PySpark
#### In this project, I have built a word count application that calculates the most common words in the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) retrieved from [Project Gutenberg]. It is based on analyzing unstructured text and could be scaled to find most common words on the Internet. 
#### Reference: [Spark's Python API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

##**Table of Contents**

<a id='Part 1: Creating a base RDD and pair RDDs'></a>
 ####[Part 1: Creating a base RDD and pair RDDs ](#-Part-1:-Creating-a-base-RDD-and-pair-RDDs-)

<a id='### ** Part 2: Counting with pair RDDs **'></a>
 ####[Part 2: Counting with pair RDDs](#-Part-2:-Counting-with-pair-RDDs-)

<a id='Part 3: Finding unique words and a mean value'></a>
####[Part 3: Finding unique words and a mean value ](#-Part-3:-Finding-unique-words-and-a-mean-value-)

<a id='Part 4: Apply word count to a file'></a>
####[Part 4: Apply word count to a file](#-Part-4:-Apply-word-count-to-a-file-)

### ** Part 1: Creating a base RDD and pair RDDs **

#### The first part involves creating a base RDD with `parallelize` and using pair RDDs to count words.

#### ** (1a) Create a base RDD **
#### This can be achieved by generating a base RDD by using a Python list and the `sc.parallelize` [parallelize()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.parallelize) method.  Then we'll print out the type of the base RDD.

In [4]:
wordsList = ['cat', 'elephant', 'rat', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)
print type(wordsRDD)

<class 'pyspark.rdd.RDD'>


#### ** (1b) Pluralize **
#### A `map()` transformation is used to add the letter 's' to each string in the base RDD I just created. A simple Python function can be used to achieve this.

In [2]:
def makePlural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  No attempt is made to follow proper
        pluralization rules.

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    return word + 's'

print makePlural('cat')

cats


#### ** (1c) Apply `makePlural` to the base RDD **
#### Now each item in the base RDD is passed into a [map()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.map) transformation that applies the `makePlural()` function to each element in the base RDD. Post that the [collect()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collect) action is called to see the transformed RDD.

In [5]:
pluralRDD = wordsRDD.map(makePlural)
print pluralRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']


#### ** (1d) Pass a `lambda` function to `map` **
#### The above result can also be obtained by passing a lambda function in the map transformation.

In [6]:
pluralLambdaRDD = wordsRDD.map(lambda word : (word + 's'))
print pluralLambdaRDD.collect()

['cats', 'elephants', 'rats', 'rats', 'cats']


#### ** (1e) Length of each word **
#### Next I use `map()` and a `lambda` function to return the number of characters in each word and collecting this result directly into a variable.

In [7]:
pluralLengths =  (pluralRDD
                 .map(lambda word: len(word))
                 .collect())
print pluralLengths

[4, 9, 4, 4, 4]


#### ** (1f) Pair RDDs **
#### In order to writing the word counting program, I create a new type of RDD, called a pair RDD. A pair RDD is an RDD where each element is a pair tuple `(k, v)` where `k` is the key and `v` is the value. In this example, I create a pair consisting of `('<word>', 1)` for each word element in the RDD where each word is is set to count 1 initially.
#### Again the pair RDD is created using the `map()` transformation with a `lambda()` function to create a new RDD.

In [8]:
wordPairs = wordsRDD.map(lambda word: (word, 1))
print wordPairs.collect()

[('cat', 1), ('elephant', 1), ('rat', 1), ('rat', 1), ('cat', 1)]


#### Now, there are multiple ways to count the number of times a particular word appears in the RDD. 

#### We could `collect()` all of the elements and count them in the driver program.  While this approach could work for small datasets, we want an approach that will work for any size dataset including terabyte- or petabyte-sized datasets. In addition, performing all of the work in the driver program is slower than performing it in parallel in the workers. For these reasons, we will use data parallel operations.

### ** Part 2: Counting with pair RDDs **

<a id='top'></a>
[back to top](#Part-1:-Creating-a-base-RDD-and-pair-RDDs-)

#### ** (2a) `groupByKey()` approach **
#### An approach you might first consider (we'll see shortly that there are better ways) is based on using the [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) transformation. As the name implies, the `groupByKey()` transformation groups all the elements of the RDD with the same key into a single list in one of the partitions. There are two problems with using `groupByKey()`:
  + #### The operation requires a lot of data movement to move all the values into the appropriate partitions.
  + #### The lists can be very large. Consider a word count of English Wikipedia: the lists for common words (e.g., the, a, etc.) would be huge and could exhaust the available memory in a worker.
 
#### Use `groupByKey()` to generate a pair RDD of type `('word', iterator)`.

In [22]:

# Note that groupByKey requires no parameters
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print '{0}: {1}'.format(key, list(value))

rat: [1, 1]
elephant: [1]
cat: [1, 1]


#### ** (2b) Use `groupByKey()` to obtain the counts **
#### Using the `groupByKey()` transformation creates an RDD containing 3 elements, each of which is a pair of a word and a Python iterator.
#### Now sum the iterator using a `map()` transformation.  The result should be a pair RDD consisting of (word, count) pairs.

In [30]:
wordCountsGrouped = wordsGrouped.map(lambda (word, x): (word, sum(x)))
print wordCountsGrouped.collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


#### ** (2c) Counting using `reduceByKey` **
#### A better approach is to start from the pair RDD and then use the [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey) transformation to create a new pair RDD. The `reduceByKey()` transformation gathers together pairs that have the same key and applies the function provided to two values at a time, iteratively reducing all of the values to a single value. `reduceByKey()` operates by applying the function first within each partition on a per-key basis and then across the partitions, allowing it to scale efficiently to large datasets.

In [35]:
# Note that reduceByKey takes in a function that accepts two values and returns a single value
wordCounts = wordPairs.reduceByKey(lambda x, y: x + y)
print wordCounts.collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


#### ** (2d) All together **
#### The expert version of the code performs the `map()` to pair RDD, `reduceByKey()` transformation, and `collect` in one statement.

In [37]:
wordCountsCollected = (wordsRDD
                       .map(lambda word: (word, 1))
                       .reduceByKey(lambda x, y: x + y)
                       .collect())
print wordCountsCollected

[('rat', 2), ('elephant', 1), ('cat', 2)]


### ** Part 3: Finding unique words and a mean value **

<a id='top'></a>
[back to top](#Part-1:-Creating-a-base-RDD-and-pair-RDDs-)

#### ** (3a) Unique words **
#### Calculate the number of unique words in `wordsRDD`.  You can use other RDDs that you have already created to make this easier.

In [43]:
uniqueWords = wordsRDD.distinct().count()
print uniqueWords

3


#### ** (3b) Mean using `reduce` **
#### Find the mean number of words per unique word in `wordCounts`.
#### Use a `reduce()` action to sum the counts in `wordCounts` and then divide by the number of unique words.  First `map()` the pair RDD `wordCounts`, which consists of (key, value) pairs, to an RDD of values.

In [49]:
from operator import add
totalCount = (wordCounts
              .map(lambda (word, count): count)
              .reduce(add))
average = totalCount / float(uniqueWords)
print totalCount
print round(average, 2)

5
1.67


### ** Part 4: Apply word count to a file **

<a id='top'></a>
[back to top](#Part-1:-Creating-a-base-RDD-and-pair-RDDs-)

#### In this section we will finish developing our word count application.  We'll have to build the `wordCount` function, deal with real world problems like capitalization and punctuation, load in our data source, and compute the word count on the new data.

#### ** (4a) `wordCount` function **
#### First, define a function for word counting.  You should reuse the techniques that have been covered in earlier parts of this lab.  This function should take in an RDD that is a list of words like `wordsRDD` and return a pair RDD that has all of the words and their associated counts.

In [26]:
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    collectWordCount =  (wordListRDD
                       .map(lambda word: (word,[1]))
                       .map(lambda (word, count): (word, sum(count)))
                       .reduceByKey(lambda x, y: x + y) 
                        )
                        
    return collectWordCount
   
print wordCount(wordsRDD).collect()

[('rat', 2), ('elephant', 1), ('cat', 2)]


#### ** (4b) Capitalization and punctuation **
#### Real world files are more complicated than the data we have been using in this lab. Some of the issues we have to address are:
  + #### Words should be counted independent of their capitialization (e.g., Spark and spark should be counted as the same word).
  + #### All punctuation should be removed.
  + #### Any leading or trailing spaces on a line should be removed.
 
#### Define the function `removePunctuation` that converts all text to lower case, removes any punctuation, and removes leading and trailing spaces.  Use the Python [re](https://docs.python.org/2/library/re.html) module to remove any text that is not a letter, number, or space. Reading `help(re.sub)` might be useful.

In [78]:
import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only spaces, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    return re.sub(r'[^\s\w]', '', text.lower().strip())
    
print removePunctuation('Hi, you!')
print removePunctuation(' No under_score!')

hi you
no under_score


#### ** (4c) Load a text file **
#### For the next part of this lab, we will use the [Complete Works of William Shakespeare](http://www.gutenberg.org/ebooks/100) from [Project Gutenberg](http://www.gutenberg.org/wiki/Main_Page). To convert a text file into an RDD, we use the `SparkContext.textFile()` method. We also apply the recently defined `removePunctuation()` function using a `map()` transformation to strip out the punctuation and change all text to lowercase.  Since the file is large we use `take(15)`, so that we only print 15 lines.

In [79]:
import os.path
baseDir = os.path.join('data')
inputPath = os.path.join('cs100', 'lab1', 'shakespeare.txt')
fileName = os.path.join(baseDir, inputPath)

shakespeareRDD = (sc
                  .textFile(fileName, 8)
                  .map(removePunctuation))
print '\n'.join(shakespeareRDD
                .zipWithIndex()  # to (line, lineNum)
                .map(lambda (l, num): '{0}: {1}'.format(num, l))  # to 'lineNum: line'
                .take(15))

0: 1609
1: 
2: the sonnets
3: 
4: by william shakespeare
5: 
6: 
7: 
8: 1
9: from fairest creatures we desire increase
10: that thereby beautys rose might never die
11: but as the riper should by time decease
12: his tender heir might bear his memory
13: but thou contracted to thine own bright eyes
14: feedst thy lights flame with selfsubstantial fuel


#### ** (4d) Words from lines **
#### Before we can use the `wordcount()` function, we have to address two issues with the format of the RDD:
  + #### The first issue is that  that we need to split each line by its spaces.
  + #### The second issue is we need to filter out empty lines.
 
#### Apply a transformation that will split each element of the RDD by its spaces. For each element of the RDD, you should apply Python's string [split()](https://docs.python.org/2/library/string.html#string.split) function. You might think that a `map()` transformation is the way to do this, but think about what the result of the `split()` function will be.

In [80]:
shakespeareWordsRDD = shakespeareRDD.flatMap(lambda x: x.split(" "))
shakespeareWordCount = shakespeareWordsRDD.count()
print shakespeareWordsRDD.top(5)
print shakespeareWordCount

[u'zwaggerd', u'zounds', u'zounds', u'zounds', u'zounds']
928908


#### ** (4e) Remove empty elements **
#### The next step is to filter out the empty elements.  Remove all entries where the word is `''`.

In [87]:
shakeWordsRDD = shakespeareWordsRDD.filter(lambda x: not re.match(r'^\s*$', x))
shakeWordCount = shakeWordsRDD.count()
print shakeWordCount

882996


#### ** (4f) Count the words **
#### We now have an RDD that is only words.  Next, let's apply the `wordCount()` function to produce a list of word counts. We can view the top 15 words by using the `takeOrdered()` action; however, since the elements of the RDD are pairs, we need a custom sort function that sorts using the value part of the pair.
#### You'll notice that many of the words are common English words. These are called stopwords. 
#### Use the `wordCount()` function and `takeOrdered()` to obtain the fifteen most common words and their counts.

In [91]:
top15WordsAndCounts = wordCount(shakeWordsRDD).takeOrdered(15, key = lambda(w, c): -c)
print '\n'.join(map(lambda (w, c): '{0}: {1}'.format(w, c), top15WordsAndCounts))

the: 27361
and: 26028
i: 20681
to: 19150
of: 17463
a: 14593
you: 13615
my: 12481
in: 10956
that: 10890
is: 9134
not: 8497
with: 7771
me: 7769
it: 7678
